
# Shark Tank

## imports

In [1]:
import pandas as pd
import seaborn as sns
import re
import sys
sys.path.append('../')

import src.date_format as dt
import src.moon_ph as m

## load data

In [2]:
sh = pd.read_csv('./attacks.csv',encoding="ISO-8859-1")
#sh.head()

## Limpiar columnas

In [3]:
#fix col names
diccio_todas = {col: col.replace(" ","_")  for col in list(sh.columns)}
sh.rename(columns=diccio_todas, inplace=True)


In [4]:
#drop unnecessary cols
sh.drop(["Unnamed:_22","Unnamed:_23", "Case_Number.1","Case_Number.2","href_formula","href","pdf"], axis=1, inplace=True)

In [5]:
sh.shape

(25723, 17)

In [6]:
#work data frame
sh.head(2)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",6302.0


## Clean the data
### NAN values


In [7]:
sh.isnull().sum()

Case_Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex_                      19986
Age                       22252
Injury                    19449
Fatal_(Y/N)               19960
Time                      22775
Species_                  22259
Investigator_or_Source    19438
original_order            19414
dtype: int64

## drop NaN rows

In [8]:
sh.dropna(axis=0, how = "all",inplace=True)

In [9]:
sh.shape

(8703, 17)

In [10]:
sh.drop_duplicates(inplace=True)

In [11]:
sh.shape

(6311, 17)

In [12]:
sh.drop_duplicates(subset=['Case_Number'],inplace=True)
sh.shape

(6288, 17)

## ask for odd cases *

In [13]:
sh.drop([6302,25722], axis=0, inplace=True)

In [14]:
sh.shape

(6286, 17)

## fill NaN values

In [15]:
sh.Year.interpolate(method='linear', limit_direction='forward', axis=0)

0       2018.0
1       2018.0
2       2018.0
3       2018.0
4       2018.0
         ...  
6297       0.0
6298       0.0
6299       0.0
6300       0.0
6301       0.0
Name: Year, Length: 6286, dtype: float64

### fix year

In [16]:
sh["Year"] = sh.Date.apply(dt.get_year)

### pick object columns  **

In [17]:
string = list(sh.select_dtypes(include="object").columns) 
print(string)

['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex_', 'Age', 'Injury', 'Fatal_(Y/N)', 'Time', 'Species_', 'Investigator_or_Source']


In [18]:
sh[string].fillna("unknown", inplace=True)
sh.tail(6)

/home/alejandro/.local/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order
6296,ND.0006,Before 1906,1906,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,NaN,FATAL,Y,NaN,Said to involve a grey nurse shark that leapt ...,"L. Becke in New York Sun, 9/9/1906; L. Schultz...",7.0
6297,ND.0005,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",6.0
6298,ND.0004,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",5.0
6299,ND.0003,1900-1905,1900,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",4.0
6300,ND.0002,1883-1889,1883,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",3.0
6301,ND.0001,1845-1853,1845,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,2.0


## done the fools way

In [19]:
sh.fillna("unkown",inplace=True)

## Invalid Type is unusable data

In [20]:
sh.Type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', 'unkown', 'Boat', 'Boatomg'], dtype=object)

In [21]:
sh = sh[sh["Type"] != "Invalid"]
sh.shape


(5743, 17)

In [22]:
sh.head(6)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,unkown,"K.McMurray, TrackingSharks.com",6302.0
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,unkown,Minor injury to lower leg,N,unkown,2 m shark,"B. Myatt, GSAF",6300.0
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,unkown,Lacerations to leg & hand shark PROVOKED INCIDENT,N,unkown,"Tiger shark, 3m",A .Kipper,6299.0
5,2018.06.03.b,03-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,unkown,"No injury, board bitten",N,unkown,unkown,"Daily Telegraph, 6/4/2018",6298.0
6,2018.06.03.a,03-Jun-2018,2018,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",6297.0


## subset fecha formato

In [23]:
sh["Date_format"] = sh.Date.apply(dt.date_r)
#sh.loc[r"\d+\-\w{3}\-\d+"].shape

In [24]:
sh.head(6)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order,Date_format
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0,25-Jun-2018
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,unkown,"K.McMurray, TrackingSharks.com",6302.0,18-Jun-2018
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,unkown,Minor injury to lower leg,N,unkown,2 m shark,"B. Myatt, GSAF",6300.0,08-Jun-2018
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,unkown,Lacerations to leg & hand shark PROVOKED INCIDENT,N,unkown,"Tiger shark, 3m",A .Kipper,6299.0,04-Jun-2018
5,2018.06.03.b,03-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,unkown,"No injury, board bitten",N,unkown,unkown,"Daily Telegraph, 6/4/2018",6298.0,03-Jun-2018
6,2018.06.03.a,03-Jun-2018,2018,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",6297.0,03-Jun-2018


## new data frame sh_f

In [25]:
sh_f = sh[sh["Date_format"] != "date not complete"]


In [27]:
sh_f.shape

(4902, 18)

## nueva columna Moon_phase
### subset?

In [29]:
sh_f["Moon_phase"] = sh_f.Date_format.apply(m.moon)

/tmp/ipykernel_6343/693218441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sh_f["Moon_phase"] = sh_f.Date_format.apply(m.moon)


In [30]:
sh_f.head(7)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order,Date_format,Moon_phase
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0,25-Jun-2018,12
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,unkown,"K.McMurray, TrackingSharks.com",6302.0,18-Jun-2018,5
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,unkown,Minor injury to lower leg,N,unkown,2 m shark,"B. Myatt, GSAF",6300.0,08-Jun-2018,25
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,unkown,Lacerations to leg & hand shark PROVOKED INCIDENT,N,unkown,"Tiger shark, 3m",A .Kipper,6299.0,04-Jun-2018,21
5,2018.06.03.b,03-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,unkown,"No injury, board bitten",N,unkown,unkown,"Daily Telegraph, 6/4/2018",6298.0,03-Jun-2018,20
6,2018.06.03.a,03-Jun-2018,2018,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",6297.0,03-Jun-2018,20
7,2018.05.27,27-May-2018,2018,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,unkown,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",6296.0,27-May-2018,13


In [35]:
sh_f.Moon_

4902